In [ ]:
from nasbench import api

In [ ]:
nasbench = api.NASBench("API/nasbench_full.tfrecord")

In [ ]:
import sys
sys.path.append("..")
import time
import numpy as np
import seaborn as sns
import importlib
import NASBench101Node
from tqdm import tqdm
from MCTS import mcts_agent
import NASBench101MCTS
from nas_bench_201 import NASBench201Node
from nas_bench_301 import NASBench301Node
importlib.reload(NASBench201Node)
importlib.reload(NASBench301Node)
importlib.reload(NASBench101Node)
importlib.reload(NASBench101MCTS)
importlib.reload(mcts_agent)
from NASBench101Node import NASBench101Node, NASBench101Cell
from NASBench101MCTS import NASBench101UCT

In [ ]:
root_node = NASBench101Node(state=NASBench101Cell(n_vertices=7))
uct = NASBench101UCT(root_node=root_node, api=nasbench, params_path="/userdata/T0259728/projets/nas/params.json", disable_tqdm=False)
uct.n_iter = 10000
returned_node, all_rewards, best_reward = uct.main_loop()
adjacency, operations = returned_node.state.operations_and_adjacency()
print(adjacency)
print(operations)

In [ ]:
import matplotlib.pyplot as plt
print(best_reward[-1])
plt.plot(best_reward)

## Counting the number of architectures we can make

In [ ]:
def play_act(node):
    available_actions = node.get_action_tuples()
    if len(available_actions) == 0:
        adjacency, operations = node.state.operations_and_adjacency()
        model_spec = ModelSpecAPI.ModelSpec(
                # Adjacency matrix of the module
                matrix=adjacency,
                # Operations at the vertices of the module, matches order of matrix
                ops=operations)
        is_valid = nasbench.is_valid(model_spec)
        if is_valid:
            return 1
        else: return 0
    for act in available_actions:
        playout_node.play_action(random_action)
        play_act(playout_node)

for i in tqdm(range(100)):
    node = NASBench101Node(state=NASBench101Cell(n_vertices=7))
    node_type = type(node)
    playout_node = node_type(state=copy.deepcopy(node.state))
    n_archi = 0
    n_


    # is_valid = False
    # while not is_valid:
    #     playout_node = node_type(state=copy.deepcopy(node.state))
    #     while not playout_node.is_terminal():
    #         available_actions = playout_node.get_action_tuples()
    #         random_action = available_actions[np.random.randint(len(available_actions))]
    #         playout_node.play_action(random_action)
    #     adjacency, operations = playout_node.state.operations_and_adjacency()
    #     model_spec = ModelSpecAPI.ModelSpec(
    #             # Adjacency matrix of the module
    #             matrix=adjacency,
    #             # Operations at the vertices of the module, matches order of matrix
    #             ops=operations)
    #     is_valid = nasbench.is_valid(model_spec)
    # adjacency, operations = playout_node.state.operations_and_adjacency()

In [ ]:
import copy 
from nasbench import api as ModelSpecAPI

n_archi = 0
n_simil = 0
list_adj = []
list_op = []
for i in tqdm(range(100000)):
    node = NASBench101Node(state=NASBench101Cell(n_vertices=7))
    is_valid = False
    while not is_valid:
        node_type = type(node)
        playout_node = node_type(state=copy.deepcopy(node.state))
        while not playout_node.is_terminal():
            available_actions = playout_node.get_action_tuples()
            random_action = available_actions[np.random.randint(len(available_actions))]
            playout_node.play_action(random_action)
        adjacency, operations = playout_node.state.operations_and_adjacency()
        model_spec = ModelSpecAPI.ModelSpec(
                # Adjacency matrix of the module
                matrix=adjacency,
                # Operations at the vertices of the module, matches order of matrix
                ops=operations)
        is_valid = nasbench.is_valid(model_spec)
    adjacency, operations = playout_node.state.operations_and_adjacency()

    # print(adjacency)
    # print(operations)
    # print(list_adj)
    # print(list_op)
    is_in_adj = False
    for adj in list_adj:
        if np.all(adjacency.flatten() == adj):
            is_in_adj = True
    is_in_op = False
    for op in list_op:
        if operations == op:
            is_in_op = True
    # print(f"Adj is in list_adj: {is_in_adj}")
    # print(f"Op is in list_op: {is_in_op}")
    if is_in_adj and is_in_op:
        n_simil += 1
    else:
        n_archi += 1
        list_adj.append(adjacency.flatten())
        list_op.append(operations)
    if i % 1000 == 0:
        print(f"We found {n_archi} different architectures with {n_simil} repeats")
        
print(f"We found {n_archi} different architectures with {n_simil} repeats")

In [ ]:
print(f"We found {n_archi} different architectures with {n_simil} repeats")

In [ ]:
INPUT = 'input'
OUTPUT = 'output'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'

In [ ]:
model_spec = api.ModelSpec(
  # Adjacency matrix of the module
  matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
          [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
          [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
          [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
          [0, 0, 0, 0, 0, 0, 0]],   # output layer
  # Operations at the vertices of the module, matches order of matrix
  ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])

# Query this model from dataset, returns a dictionary containing the metrics
# associated with this model.
print('Querying an Inception-like model.')
data = nasbench.query(model_spec)
print(data)
print(nasbench.get_budget_counters())   # prints (total time, total epochs)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
print('\nIterating over unique models in the dataset.')
columns=['module_adjacency',
 'module_operations',
 'trainable_parameters',
 'halfway_training_time',
 'halfway_train_accuracy',
 'halfway_validation_accuracy',
 'halfway_test_accuracy',
 'final_training_time',
 'final_train_accuracy',
 'final_validation_accuracy',
 'final_test_accuracy']
df = pd.DataFrame(columns=columns)
i = 0
for unique_hash in tqdm(nasbench.hash_iterator()):
    fixed_metrics, computed_metrics = nasbench.get_metrics_from_hash(
        unique_hash)
    dic = {}
    for k, v in fixed_metrics.items():
        dic[k] = v
    mets= computed_metrics[108]
    for col_name in  ['halfway_training_time', 'halfway_train_accuracy', 'halfway_validation_accuracy', 
                      'halfway_test_accuracy', 'final_training_time', 'final_train_accuracy', 
                      'final_validation_accuracy', 'final_test_accuracy']:
        dic[col_name] = np.mean([e[col_name] for e in mets])
    # For demo purposes, break here instead of iterating through whole set.
    df = df.append(pd.Series(dic), ignore_index=True)
    if i % 100000 == 0:
        df.to_csv("nas_bench_101.csv")
    i += 1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv("nas_bench_101.csv")
# f, ax = plt.subplots(figsize=(10,10))
# sns.scatterplot(data=df, x="trainable_parameters", y="final_test_accuracy", s=20)

In [ ]:
df2 = df.sort_values(["final_test_accuracy"], ascending=False)
id = 0
print(df2["module_operations"].iloc[id])
print(df2["final_test_accuracy"].iloc[id])
print(df2["module_adjacency"].iloc[id])

In [ ]:
rd_index = np.random.randint(df.shape[0])
print(df["module_adjacency"].iloc[rd_index])
print(df2["final_test_accuracy"].iloc[rd_index])
df["module_operations"].iloc[rd_index]

In [ ]:
import sys
sys.path.append("..")
import time
import numpy as np
import seaborn as sns
import importlib
import MCTS.mcts_agent
import NASBench201MCTS
importlib.reload(MCTS.mcts_agent)
importlib.reload(NASBench201MCTS)
sns.set_context("notebook")
sns.set_theme(sns.plotting_context("notebook", font_scale=1), style="whitegrid", palette=sns.color_palette("deep"))
from MCTS.mcts_agent import MCTSAgent, UCT
from NASBench201MCTS import NASBench201UCT, NASBench201RAVE, NASBench201GRAVE
from nas_201_api import NASBench201API as API

In [ ]:
t1 = time.time()
api = API('/userdata/T0259728/Bureau/NAS-Bench-201-v1_1-096897.pth', verbose=False)
t2 = time.time()
print(f"API inistialized in {t2-t1} seconds.")

In [ ]:
from NASBench201Node import NASBench201Node, NASBench201AMAFNode, NASBench201Cell

In [ ]:
N_RUNS = 20

In [ ]:
avg_rewards_uct = []
avg_best_reward_uct = []

for n_run in range(N_RUNS):
    print(f"[{n_run}/{N_RUNS}]")        
    root_node = NASBench201Node(state=NASBench201Cell(4))
    uct = NASBench201UCT(root_node, api,
                      params_path="/userdata/T0259728/projets/nas/params.json", disable_tqdm=True)
    uct.C = 0.1
    uct.n_iter=500
    uct.playouts_per_selection = 1
    returned_node, all_rewards, best_reward = uct.main_loop()
    avg_rewards_uct.append(all_rewards)
    avg_best_reward_uct.append(best_reward)

In [ ]:
avg_rewards_rave = []
avg_best_reward_rave = []

for n_run in range(N_RUNS):
    try: 
        print(f"{n_run}/{N_RUNS}")
        root_node = NASBench201AMAFNode(state=NASBench201Cell(4))
        rave = NASBench201RAVE(root_node, api,
                          params_path="/userdata/T0259728/projets/nas/params.json", disable_tqdm=True)
        rave.C = 0.1
        rave.b=0.1
        rave.n_iter=500
        rave.playouts_per_selection = 1
        returned_node, all_rewards, best_reward = rave.main_loop()
        avg_rewards_rave.append(all_rewards)
        avg_best_reward_rave.append(best_reward)
    except Exception:
        continue

In [ ]:
avg_rewards_grave = []
avg_best_reward_grave = []

for n_run in range(N_RUNS):
    try: 
        print(f"{n_run}/{N_RUNS}")
        root_node = NASBench201AMAFNode(state=NASBench201Cell(4))
        grave = NASBench201GRAVE(root_node, api,
                          params_path="/userdata/T0259728/projets/nas/params.json", disable_tqdm=True)
        grave.C = 0.1
        grave.b=0.1
        grave.ref = 10
        grave.n_iter=500
        grave.playouts_per_selection = 1
        returned_node, all_rewards, best_reward = grave.main_loop()
        avg_rewards_grave.append(all_rewards)
        avg_best_reward_grave.append(best_reward)
    except Exception:
        continue

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.mean(np.array(avg_be st_reward_uct), axis=0), label="UCT")
plt.plot(np.mean(np.array(avg_best_reward_rave), axis=0), label="RAVE")
plt.plot(np.mean(np.array(avg_best_reward_grave), axis=0), label="GRAVE")
plt.title("Evolution of the best architecture reward \n 400 iterations"); plt.xlabel("Iteration"); plt.ylabel("CIFAR-100 accuracy"); plt.legend();
plt.ylim([0.56,0.62]);
# plt.savefig("../figures/uct_vs_rave_vs_grave_best_reward_400.png")
plt.show()

In [ ]:
importlib.reload(MCTS.mcts_agent)
importlib.reload(NASBench201MCTS)
from NASBench201MCTS import NASBench201UCT_NTK
import tensorly as tl
tl.set_backend("pytorch")

In [ ]:
avg_rewards_uct_ntk = []
avg_best_reward_uct_ntk = []
avg_accuracies_uct_ntk = []
avg_best_accuracy_uct_ntk = []

for n_run in range(N_RUNS):
    print(f"[{n_run}/{N_RUNS}]")        
    root_node = NASBench201Node(state=NASBench201Cell(4))
    ntk = NASBench201UCT_NTK(root_node, api,
                      params_path="/userdata/T0259728/projets/nas/params.json", disable_tqdm=False)
    ntk.C = 0.1
    ntk.n_iter=400
    ntk.playouts_per_selection = 1
    returned_node, all_rewards, best_reward, all_accuracies, best_accuracies = ntk.main_loop()
    avg_rewards_uct_ntk.append(all_rewards)
    avg_best_reward_uct_ntk.append(best_reward)
    avg_accuracies_uct_ntk.append(all_accuracies)
    avg_best_accuracy_uct_ntk.append(best_accuracies)

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(np.mean(np.array(avg_best_reward_uct), axis=0), label="UCT")
f, ax1 = plt.subplots()
ax1.plot(np.mean(np.array(avg_best_reward_uct_ntk), axis=0), label="NTK Metric", color="#3d405b")
ax1.set_ylabel("NTK Metric")

ax2 = ax1.twinx()
ax2.plot(np.mean(np.array(avg_best_accuracy_uct_ntk), axis=0), label="Accuracy", color="#e07a5f")
ax2.set_ylabel("CIFAR-100 accuracy")

plt.title("Evolution of the best architecture reward \n 400 iterations"); plt.xlabel("Iteration"); plt.ylabel("CIFAR-100 accuracy"); 
plt.legend();
# plt.ylim([0.56,0.62]);
plt.savefig("../figures/ntk_metric.png")
plt.show()

In [ ]:
plt.plot(np.mean(np.array(avg_best_reward_uct_ntk), axis=0), label="kzjnj")

In [ ]:
avg_best_reward_uct_ntk